In [5]:
!pip install jsonlines

In [6]:
# train data를 type 1과 type 2로 구분

import jsonlines
import json

input_file_path = 'train.json'
train_file_path = 'train_type1.jsonl'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with jsonlines.open(train_file_path, mode='w') as writer:
    for item in data['dataset']:
      if(item['type']=='Type 1') :
        writer.write(item)

train_file_path = 'train_type2.jsonl'

with jsonlines.open(train_file_path, mode='w') as writer:
    for item in data['dataset']:
      if(item['type']=='Type 2') :
        writer.write(item)

In [ ]:
# train data에서 type 1, 2인 데이터가 각각 몇 개씩 있는지 구한다.

with open('train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

n_type1=0
n_type2=0

with jsonlines.open(eval_file_path, mode='w') as writer:
    for item in data['dataset']:
      if(item['type']=='Type 1') :
        n_type1+=1
      else : n_type2+=1

print(n_type1, n_type2)

13448 16552


In [ ]:
# test.json에 test.jsonl을 생성

import jsonlines
import json

input_file_path = 'test.json'
test_file_path = 'test.jsonl'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with jsonlines.open(test_file_path, mode='w') as writer:
    for item in data['dataset']:
        writer.write(item)

In [ ]:
# test_dataset을 생성하고, size와 set의 단어 개수가 다른 데이터가 몇 개인지 구한다.

from datasets import load_dataset

test_dataset = load_dataset('json', data_files='test.jsonl', split='train')

total=1500 # total number of test data
num=0
for i in test_dataset :
  num+=len(i['set'].split('#'))==i['size']

print(num)

1000

In [ ]:
# 3만 개의 train data에서 3천 개의 valid data를 추출
# valid data에서 type 1, 2인 데이터의 비율은 3 : 1
# 그러므로 type 1, 2인 valid data는 각각 2250, 750개

from datasets import load_dataset
from sklearn.model_selection import train_test_split

type1_dataset = load_dataset('json', data_files='train_type1.jsonl', split='train') # 2250 sampling
type2_dataset = load_dataset('json', data_files='train_type2.jsonl', split='train') # 750 sampling

type1_train, type1_valid = train_test_split(type1_dataset, test_size=2250, random_state=123)
type2_train, type2_valid = train_test_split(type2_dataset, test_size=750, random_state=123)

for key, value in type2_train.items():
    if key in type1_train:
        type1_train[key].extend(value)
    else:
        type1_train[key] = value

for key, value in type2_valid.items():
    if key in type1_valid:
        type1_valid[key].extend(value)
    else:
        type1_valid[key] = value

split_train_tmp=type1_train
split_valid_tmp=type1_valid

In [ ]:
split_train=[]

for i in range(27000) :
   split_train.append({"id":split_train_tmp['id'][i], "type":split_train_tmp['type'][i], "size":split_train_tmp['size'][i], "set":split_train_tmp['set'][i], "sentence":split_train_tmp['sentence'][i]})

In [ ]:
split_valid=[]

for i in range(3000) :
  split_valid.append({"id":split_valid_tmp['id'][i], "type":split_valid_tmp['type'][i], "size":split_valid_tmp['size'][i], "set":split_valid_tmp['set'][i], "sentence":split_valid_tmp['sentence'][i]})

In [ ]:
with open('split_train.json', 'w', encoding='UTF-8') as f :
	json.dump(split_train, f, indent=2, ensure_ascii=False)

In [ ]:
with open('split_valid.json', 'w', encoding='UTF-8') as f :
	json.dump(split_valid, f, indent=2, ensure_ascii=False)

In [ ]:
import random

sampled_valid=random.sample(split_valid, 1000)

num=0

for i in split_valid :
  if(i in sampled_valid) :
    word_list=i['set'].split('#')
    length=len(word_list)
    if(length==1) :
      continue
    target_index=random.randint(0, length-1)
    del word_list[target_index]
    new_word_list=""
    for j in word_list :
      new_word_list+=j+'#'
    i['set']=new_word_list[:-1]
    num+=1

1000

In [ ]:
with open('complete_split_valid.json', 'w', encoding='UTF-8') as f :
	json.dump(split_valid, f, indent=2, ensure_ascii=False)

In [ ]:
import random

word_infer_train=[]

for i in split_train :
  word_list=i['set'].split('#')
  length=len(word_list)
  if(length==1) :
    continue
  target_index=random.randint(0, length-1)

  word=word_list[target_index]
  del word_list[target_index]

  word_infer_train.append({"input":word_list, "output":word})

In [ ]:
with open('word_infer_train.json', 'w', encoding='UTF-8') as f :
	json.dump(word_infer_train, f, indent=2, ensure_ascii=False)